# 04 - Construction du Dashboard Streamlit

Ce notebook ne s'exécute pas directement. Il documente les étapes de création du fichier `dashboard.py`.

### Objectifs :
1. Structure de base d'une app Streamlit.
2. Chargement des données réelles.
3. Calcul des KPIs dynamiques.
4. Intégration des graphiques Plotly.
5. Intégration de l'IA.

## Étape 1 : Structure Minimale

Créez un fichier `dashboard.py`. On commence par les imports essentiels.

In [ ]:
import streamlit as st
import pandas as pd
import plotly.express as px
import os
import google.generativeai as genai
from dotenv import load_dotenv

# Configuration de la page
st.set_page_config(page_title="Dashboard Avis", layout="wide")

def main():
    st.title("Analyse des Avis Stores 📱")
    st.write("Bienvenue sur le dashboard d'analyse !")

if __name__ == "__main__":
    main()

> **Action** : Lancez `streamlit run dashboard.py` dans votre terminal pour voir le résultat.

## Étape 2 : Chargement des Données

On utilise le fichier `df_final.csv` généré dans le notebook 1.

In [ ]:
@st.cache_data
def load_data():
    if not os.path.exists('df_final.csv'):
        return pd.DataFrame()
    df = pd.read_csv('df_final.csv')
    return df

# Dans main():
df = load_data()
if df.empty:
    st.warning("Aucune donnée trouvée. Avez-vous lancés le notebook 01 ?")
else:
    st.write(f"✅ {len(df)} avis chargés con succès.")

## Étape 3 : Affichage des KPIs Réels

Plus de chiffres au hasard. On calcule les vrais indicateurs à partir du DataFrame.

In [ ]:
# Calculs
total_avis = len(df)
note_moyenne = round(df['rating'].mean(), 2)
# Calcul simple du NPS (Promoteurs - Détracteurs) / Total * 100
promoteurs = len(df[df['rating'] >= 4])
detracteurs = len(df[df['rating'] <= 2])
nps = round(((promoteurs - detracteurs) / total_avis) * 100)

# Affichage en colonnes
k1, k2, k3 = st.columns(3)
k1.metric("Total Avis", total_avis)
k2.metric("Note Moyenne", f"{note_moyenne}/5")
k3.metric("NPS", nps)

## Étape 4 : Ajout des Graphiques

On réutilise la logique du Notebook 02 pour afficher nos graphiques avec Plotly.

In [ ]:
g1, g2 = st.columns(2)
# Graphique 1 : Répartition par Source
fig_source = px.pie(df, names='source', title='Répartition Google vs Apple', color_discrete_sequence=['#36A2EB', '#FF6384'])
g1.plotly_chart(fig_source, width="stretch")

# Graphique 2 : Distribution des Notes
rating_counts = df['rating'].value_counts().sort_index().reset_index()
rating_counts.columns = ['Note', 'Volume']
fig_rating = px.bar(rating_counts, x='Volume', y='Note', orientation='h', title='Distribution des Notes')
g2.plotly_chart(fig_rating, width="stretch")

## Étape 5 : Intégration de l'IA (Réelle)

On branche Gemini pour analyser les avis affichés. Assurez-vous d'avoir votre fichier `.env`.

In [ ]:
# Chargement de la clé API
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)

st.markdown("---")
st.subheader("🧠 Analyse IA")

if st.button("Lancer l'analyse Gemini"):
    with st.spinner("L'IA analyse les avis..."):
        # Préparation d'un petit échantillon de texte pour ne pas exploser le quota
        sample_text = ""
        for i, row in df.head(100).iterrows():
            sample_text += f"- Note {row['rating']}/5 : {row['text']}\n"
        
        # Appel API
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content(f"Fais une synthèse courte de ces avis :\n{sample_text}")
        
        st.success("Analyse terminée !")
        st.info(response.text)


## Étape 6 : Styling CSS

Optionnel mais recommandé pour le look.

In [ ]:
def local_css():
    st.markdown("""
    <style>
        .stMetric { background-color: blue; padding: 15px; border-radius: 10px; }
    </style>
    """, unsafe_allow_html=True)

local_css()

## Conclusion

Vous avez maintenant un dashboard fonctionnel connecté à vos données réelles. Bravo !